In [1]:
# Check to see if we're running in Colab (versus local server)
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

We're running Colab


In [21]:
if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/drive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)
   # Switch to the target directory on the Google Drive
  import os
  drive_root = mount + "/MyDrive/Data_Glacier_Virtual_Intern"
  # Create drive_root if it doesn't exist
  create_drive_root = True
  if create_drive_root:
    print("\nColab: making sure ", drive_root, " exists.")
    os.makedirs(drive_root, exist_ok=True)
  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root

Colab: mounting Google drive on  /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Colab: making sure  /content/drive/MyDrive/Data_Glacier_Virtual_Intern  exists.

Colab: Changing directory to  /content/drive/MyDrive/Data_Glacier_Virtual_Intern
/content/drive/MyDrive/Data_Glacier_Virtual_Intern


In [22]:
# clone repository from Github
username = 'BattaLiu'
repository = 'DSVIICODE'
git_token = ''
! git clone https://{git_token}@github.com/{username}/{repository}

Cloning into 'DSVIICODE'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 22 (delta 7), reused 11 (delta 4), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [25]:
%cd $drive_root/{repository}

/content/drive/MyDrive/Data_Glacier_Virtual_Intern/DSVIICODE


In [ ]:
# # Let's load the working directory from our Git repo !
# repo_url = "https://github.com/BattaLiu/DSVIICODE.git"

# # Clone the repo unless it is already there
# repo_root = os.path.join(drive_root,"DSVIICODE")

# # Clone or refresh from the repo depending on whether we've cloned before
# if not os.path.exists( repo_root ):
#   print("Cloning from Git url: ", repo_url)
#   ! git clone $repo_url
# else:
#   print("Refreshing from Git url: ", repo_url)
#   %cd $repo_root
#   ! git pull

# print("\nChange working directory to repo root")
# %cd $repo_root

Refreshing from Git url:  https://github.com/BattaLiu/DSVIICODE.git
/content/gdrive/My Drive/Data_Glacier_Virtual_Intern/DSVIICODE
Already up to date.

Change working directory to repo root
/content/gdrive/My Drive/Data_Glacier_Virtual_Intern/DSVIICODE


In [26]:
! git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [27]:
! git config --global user.email "liubatta@gmail.com"
! git config --global user.name "BattaLiu"

In [20]:
! git add test_ingestion.ipynb
! git commit -m "git clone/pull and then move in current notebook to replace old test_ingestion.ipynb"
! git push https://{git_token}@github.com/{username}/{repository}

[main 7c34802] git clone/pull and then move in current notebook to replace old test_ingestion.ipynb
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite test_ingestion.ipynb (89%)
Counting objects: 32, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (31/31), done.
error: RPC failed; curl 55 Failed sending data to the peer
fatal: The remote end hung up unexpectedly
Writing objects: 100% (32/32), 1.66 GiB | 18.70 MiB/s, done.
Total 32 (delta 19), reused 1 (delta 0)
fatal: The remote end hung up unexpectedly
Everything up-to-date


In [ ]:
#  # Connect to Kaggle account to download a data set greater than 2GB into a seperate repository with Kaggle API token
#  ! pip install kaggle
#  os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"
#  %cd /content/drive/MyDrive/Kaggle
#  ! kaggle datasets download yanivaknin/fafdata -f issue_cmds.mar22.csv
#  # then move the dataset file into the working directory manually and change back to working dirctory
#  %cd $drive_root

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/Kaggle
 98% 360M/368M [00:03<00:00, 92.4MB/s]
100% 368M/368M [00:03<00:00, 105MB/s] 
/content/drive/MyDrive/Data_Glacier_Virtual_Intern/DSVIICODE



## Task:

* Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

* Read the file ( Present approach of reading the file )

* Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational     efficiency

* Perform basic validation on data columns : eg: remove special character , white spaces from the col name

* As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of   
  read and write file, column name in YAML

* Validate number of columns and column name of ingested file with YAML.

* Write the file in pipe separated text file (|) in gz format.

* Create a summary of the file:

    Total number of rows,

    total number of columns

    file size
    
# Data Ingestion sample code walkthrough

## 
  > Create a utility file
  
  > Config file creation
  
  > Data ingestion pipeline


In [52]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


### Write YAML file

In [53]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: issue_cmds.mar22
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - id
    - start_time
    - offset_ms

Overwriting file.yaml


In [54]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [55]:
config_data['file_name']

'issue_cmds.mar22'

In [56]:
#inspecting data of config file
config_data

{'columns': ['id', 'start_time', 'offset_ms'],
 'dataset_name': 'testfile',
 'file_name': 'issue_cmds.mar22',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'edsurv'}

In [57]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("issue_cmds.mar22.csv",delimiter=',')
df_sample.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,start_time,offset_ms,player,type,units_number,blueprint,position_x,position_y,command_name,faf_player_id,login,rating,faction,map_name,map_thumbnail,map_width,map_height
0,16630555,2022-03-25 09:22:13 UTC,654500,1,issue,2,NaN,43.933487,126.474571,TransportUnloadSpecificUnits,82362,Exselsior,1527,Cybran,Arctic Refuge - FAF version,https://content.faforever.com/maps/previews/la...,512,512
1,16534378,2022-03-11 20:19:06 UTC,993900,0,issue,52,NaN,310.636597,238.775345,FormMove,156219,StormLantern,1776,Cybran,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
2,16553544,2022-03-14 00:27:49 UTC,894900,0,issue,1,NaN,NaN,NaN,Stop,304205,Saphed,1501,Seraphim,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
3,16553544,2022-03-14 00:27:49 UTC,814400,1,issue,1,NaN,NaN,NaN,Stop,287686,demonstreamer666,1552,Cybran,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
4,16650721,2022-03-27 14:52:40 UTC,1988600,0,issue,1,NaN,361.272095,766.394592,TransportUnloadUnits,189718,ZXCCURSED,2216,Seraphim,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024


In [ ]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [1]:
#validate the header of the file
util.col_header_val(df,config_data)

NameError: ignored

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['id', 'start_time', 'offset_ms', 'player', 'type', 'units_number',
       'blueprint', 'position_x', 'position_y', 'command_name',
       'faf_player_id', 'login', 'rating', 'faction', 'map_name',
       'map_thumbnail', 'map_width', 'map_height'],
      dtype='object')
columns of YAML are: ['id', 'start_time', 'offset_ms']


In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['map_width', 'blueprint', 'map_thumbnail', 'units_number', 'position_y', 'type', 'faf_player_id', 'map_name', 'login', 'faction', 'player', 'map_height', 'rating', 'command_name', 'position_x']
Following YAML columns are not in the file uploaded []
validation failed


In [ ]:
pd.read_csv("./issue_cmds.mar22.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,start_time,offset_ms,player,type,units_number,blueprint,position_x,position_y,command_name,faf_player_id,login,rating,faction,map_name,map_thumbnail,map_width,map_height
0,16630555,2022-03-25 09:22:13 UTC,654500,1,issue,2,NaN,43.933487,126.474571,TransportUnloadSpecificUnits,82362,Exselsior,1527,Cybran,Arctic Refuge - FAF version,https://content.faforever.com/maps/previews/la...,512,512
1,16534378,2022-03-11 20:19:06 UTC,993900,0,issue,52,NaN,310.636597,238.775345,FormMove,156219,StormLantern,1776,Cybran,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
2,16553544,2022-03-14 00:27:49 UTC,894900,0,issue,1,NaN,NaN,NaN,Stop,304205,Saphed,1501,Seraphim,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
3,16553544,2022-03-14 00:27:49 UTC,814400,1,issue,1,NaN,NaN,NaN,Stop,287686,demonstreamer666,1552,Cybran,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
4,16650721,2022-03-27 14:52:40 UTC,1988600,0,issue,1,NaN,361.272095,766.394592,TransportUnloadUnits,189718,ZXCCURSED,2216,Seraphim,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10649312,16463505,2022-03-01 03:33:37 UTC,743200,1,issue,3,NaN,284.151703,341.428772,AggressiveMove,219084,Conorach,1411,UEF,TAG Craftious Maximus - FAF version,https://content.faforever.com/maps/previews/la...,512,512
10649313,16465285,2022-03-01 14:09:47 UTC,793700,0,issue,1,NaN,217.667480,175.959106,AggressiveMove,304205,Saphed,1468,Cybran,TAG Craftious Maximus - FAF version,https://content.faforever.com/maps/previews/la...,512,512
10649314,16463505,2022-03-01 03:33:37 UTC,432100,1,issue,1,NaN,354.588531,447.169708,AggressiveMove,219084,Conorach,1411,UEF,TAG Craftious Maximus - FAF version,https://content.faforever.com/maps/previews/la...,512,512
10649315,16465285,2022-03-01 14:09:47 UTC,612600,1,issue,2,NaN,418.130646,367.067932,AggressiveMove,174608,MAPTOC,1202,Cybran,TAG Craftious Maximus - FAF version,https://content.faforever.com/maps/previews/la...,512,512


In [ ]:
df

,id,start_time,offset_ms,player,type,units_number,blueprint,position_x,position_y,command_name,faf_player_id,login,rating,faction,map_name,map_thumbnail,map_width,map_height
0,16630555,2022-03-25 09:22:13 UTC,654500,1,issue,2,NaN,43.933487,126.474571,TransportUnloadSpecificUnits,82362,Exselsior,1527,Cybran,Arctic Refuge - FAF version,https://content.faforever.com/maps/previews/la...,512,512
1,16534378,2022-03-11 20:19:06 UTC,993900,0,issue,52,NaN,310.636597,238.775345,FormMove,156219,StormLantern,1776,Cybran,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
2,16553544,2022-03-14 00:27:49 UTC,894900,0,issue,1,NaN,NaN,NaN,Stop,304205,Saphed,1501,Seraphim,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
3,16553544,2022-03-14 00:27:49 UTC,814400,1,issue,1,NaN,NaN,NaN,Stop,287686,demonstreamer666,1552,Cybran,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
4,16650721,2022-03-27 14:52:40 UTC,1988600,0,issue,1,NaN,361.272095,766.394592,TransportUnloadUnits,189718,ZXCCURSED,2216,Seraphim,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10649312,16463505,2022-03-01 03:33:37 UTC,743200,1,issue,3,NaN,284.151703,341.428772,AggressiveMove,219084,Conorach,1411,UEF,TAG Craftious Maximus - FAF version,https://content.faforever.com/maps/previews/la...,512,512
10649313,16465285,2022-03-01 14:09:47 UTC,793700,0,issue,1,NaN,217.667480,175.959106,AggressiveMove,304205,Saphed,1468,Cybran,TAG Craftious Maximus - FAF version,https://content.faforever.com/maps/previews/la...,512,512
10649314,16463505,2022-03-01 03:33:37 UTC,432100,1,issue,1,NaN,354.588531,447.169708,AggressiveMove,219084,Conorach,1411,UEF,TAG Craftious Maximus - FAF version,https://content.faforever.com/maps/previews/la...,512,512
10649315,16465285,2022-03-01 14:09:47 UTC,612600,1,issue,2,NaN,418.130646,367.067932,AggressiveMove,174608,MAPTOC,1202,Cybran,TAG Craftious Maximus - FAF version,https://content.faforever.com/maps/previews/la...,512,512


In [ ]:
# import gzip
# f_in = open("./issue_cmds.mar22.csv")
# f_out = gzip.open('./issue_cmds.mar22.csv.gz', 'wb')
# f_out.writelines(f_in)
# f_out.close()
# f_in.close()

In [ ]:
df.to_csv("issue_cmds.mar22.csv.gz", 
           sep = '|',
           compression="gzip")

In [ ]:
! git status

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   file.yaml
	modified:   test_ingestion.ipynb
	modified:   testutility.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	__pycache__/
	issue_cmds.mar22.csv
	issue_cmds.mar22.csv.gz

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
! git add file.yaml
! git add test_ingestion.ipynb
! git add testutility.py


In [ ]:
! git rm issue_cmds.mar22.csv.gz

rm 'issue_cmds.mar22.csv.gz'


In [ ]:
! git commit -m 'compressed dataframe into gzip file with pipe as delimiter'

[main bb166dd] compressed dataframe into gzip file with pipe as delimiter
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
! git push

Counting objects: 14, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (13/13), done.
error: RPC failed; curl 55 Failed sending data to the peer
fatal: The remote end hung up unexpectedly
Writing objects: 100% (14/14), 1.66 GiB | 18.03 MiB/s, done.
Total 14 (delta 7), reused 1 (delta 0)
fatal: The remote end hung up unexpectedly
Everything up-to-date


In [ ]:
# import gzip
# import shutil
# with open("./issue_cmds.mar22.csv", 'rb') as f_in:
#     with gzip.open('./issue_cmds.mar22.csv.gz', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)